In [2]:
from functions import *
import pandas as pd
import streamlit as st
import folium
from streamlit_folium import folium_static

In [3]:
url_robberies = ("https://github.com/LuisHRF/Home-Insurance-Study-Project-/raw/main/01002%20(1).xlsx")

data_raw_robberies = pd.read_excel(url_robberies)

data_raw_robberies.head(10)


,,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos
0,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total Nacional,226761.0,118125.0,665622.0,272638.0,104988.0,84721.0,35072.0,64711.0,44406.0,3844.0,8238.0
2,Araba/Álava,1386.0,745.0,4184.0,1099.0,8.0,434.0,178.0,114.0,82.0,6.0,14.0
3,Albacete,1638.0,851.0,3771.0,1784.0,421.0,636.0,204.0,211.0,120.0,23.0,46.0
4,Alicante/Alacant,11416.0,5326.0,27014.0,12716.0,3723.0,6275.0,1241.0,1997.0,1161.0,245.0,343.0
5,Almería,4343.0,2517.0,6437.0,4193.0,1106.0,1602.0,399.0,657.0,397.0,67.0,81.0
6,Ávila,722.0,453.0,784.0,646.0,79.0,269.0,75.0,26.0,10.0,7.0,5.0
7,Badajoz,2516.0,1621.0,4844.0,2682.0,622.0,967.0,383.0,231.0,107.0,29.0,70.0
8,Balears (Illes),10159.0,4495.0,23009.0,6806.0,3195.0,1937.0,738.0,1519.0,994.0,82.0,228.0
9,Barcelona,16470.0,10527.0,141789.0,44688.0,22066.0,12042.0,5442.0,22792.0,17358.0,808.0,1787.0


In [4]:
data_clean = data_raw_robberies.dropna()

data_clean 

,,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos
1,Total Nacional,226761.0,118125.0,665622.0,272638.0,104988.0,84721.0,35072.0,64711.0,44406.0,3844.0,8238.0
2,Araba/Álava,1386.0,745.0,4184.0,1099.0,8.0,434.0,178.0,114.0,82.0,6.0,14.0
3,Albacete,1638.0,851.0,3771.0,1784.0,421.0,636.0,204.0,211.0,120.0,23.0,46.0
4,Alicante/Alacant,11416.0,5326.0,27014.0,12716.0,3723.0,6275.0,1241.0,1997.0,1161.0,245.0,343.0
5,Almería,4343.0,2517.0,6437.0,4193.0,1106.0,1602.0,399.0,657.0,397.0,67.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...
779,Zaragoza,5171.0,1988.0,14860.0,7904.0,1895.0,1756.0,1271.0,1531.0,1052.0,96.0,235.0
780,Ceuta,833.0,393.0,983.0,723.0,497.0,93.0,49.0,189.0,141.0,8.0,13.0
781,Melilla,1011.0,501.0,1043.0,774.0,427.0,156.0,79.0,194.0,171.0,1.0,8.0
782,En el extranjero,158.0,17.0,2427.0,674.0,142.0,19.0,51.0,177.0,138.0,5.0,6.0


In [5]:
unique_first_column = data_clean.iloc[:, 0].unique()

unique_first_column 

array(['    Total Nacional', '    Araba/Álava', '    Albacete',
       '    Alicante/Alacant', '    Almería', '    Ávila', '    Badajoz',
       '    Balears (Illes)', '    Barcelona', '    Burgos',
       '    Cáceres', '    Cádiz', '    Castellón/Castelló',
       '    Ciudad Real', '    Córdoba', '    Coruña (A)', '    Cuenca',
       '    Girona', '    Granada', '    Guadalajara', '    Gipuzkoa',
       '    Huelva', '    Huesca', '    Jaén', '    León', '    Lleida',
       '    Rioja (La)', '    Lugo', '    Madrid', '    Málaga',
       '    Murcia', '    Navarra', '    Ourense', '    Asturias',
       '    Palencia', '    Palmas (Las)', '    Pontevedra',
       '    Salamanca', '    Santa Cruz de Tenerife', '    Cantabria',
       '    Segovia', '    Sevilla', '    Soria', '    Tarragona',
       '    Teruel', '    Toledo', '    Valencia/València',
       '    Valladolid', '    Bizkaia', '    Zamora', '    Zaragoza',
       '    Ceuta', '    Melilla', '    En el extranjero',
   

In [6]:
# Calculate the variable block sizes

block_sizes = [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]
start = 1
year_ranges = {}

for i, size in enumerate(block_sizes):
    end = start + size - 1
    year = 2023 - i
    year_ranges[(start, end)] = year
    start = end + 2  

print(year_ranges)

{(1, 55): 2023, (57, 111): 2022, (113, 167): 2021, (169, 223): 2020, (225, 279): 2019, (281, 335): 2018, (337, 391): 2017, (393, 447): 2016, (449, 503): 2015, (505, 559): 2014, (561, 615): 2013, (617, 671): 2012, (673, 727): 2011, (729, 783): 2010}


In [7]:
year_range = {
    (1, 55): 2023,
    (57, 111): 2022,
    (113, 167): 2021,
    (169, 223): 2020,
    (225, 279): 2019,
    (281, 335): 2018,
    (337, 391): 2017,
    (393, 447): 2016,
    (449, 503): 2015,
    (505, 559): 2014,
    (561, 615): 2013,
    (617, 671): 2012,
    (673, 727): 2011,
    (729, 783): 2010
}

data_clean_years = defi_years_per_block(data_clean, year_range)

data_clean_years.tail(10)

,,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos,Year
774,Toledo,4301.0,1476.0,5420.0,7131.0,487.0,2313.0,1175.0,398.0,229.0,30.0,90.0,2010.0
775,Valencia/València,17646.0,6380.0,44641.0,32568.0,9350.0,9451.0,3897.0,6345.0,4505.0,388.0,774.0,2010.0
776,Valladolid,3183.0,1121.0,7937.0,2993.0,652.0,565.0,506.0,301.0,180.0,23.0,64.0,2010.0
777,Bizkaia,6945.0,2849.0,15197.0,10849.0,29.0,2336.0,1470.0,2045.0,1449.0,157.0,261.0,2010.0
778,Zamora,810.0,340.0,1090.0,752.0,93.0,165.0,205.0,62.0,33.0,7.0,18.0,2010.0
779,Zaragoza,5171.0,1988.0,14860.0,7904.0,1895.0,1756.0,1271.0,1531.0,1052.0,96.0,235.0,2010.0
780,Ceuta,833.0,393.0,983.0,723.0,497.0,93.0,49.0,189.0,141.0,8.0,13.0,2010.0
781,Melilla,1011.0,501.0,1043.0,774.0,427.0,156.0,79.0,194.0,171.0,1.0,8.0,2010.0
782,En el extranjero,158.0,17.0,2427.0,674.0,142.0,19.0,51.0,177.0,138.0,5.0,6.0,2010.0
783,Desconocida,6.0,5.0,188.0,218.0,0.0,3.0,30.0,5.0,2.0,1.0,0.0,2010.0


In [8]:
if data_clean_years.columns[0] == 'Unnamed: 0':
    data_clean_years = data_clean_years.rename(columns={data_clean_years.columns[0]: 'Province'})
else:
    data_clean_years = data_clean_years.rename(columns={data_clean_years.columns[0]: 'Province'})

data_clean_years

,Province,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos,Year
1,Total Nacional,226761.0,118125.0,665622.0,272638.0,104988.0,84721.0,35072.0,64711.0,44406.0,3844.0,8238.0,2023.0
2,Araba/Álava,1386.0,745.0,4184.0,1099.0,8.0,434.0,178.0,114.0,82.0,6.0,14.0,2023.0
3,Albacete,1638.0,851.0,3771.0,1784.0,421.0,636.0,204.0,211.0,120.0,23.0,46.0,2023.0
4,Alicante/Alacant,11416.0,5326.0,27014.0,12716.0,3723.0,6275.0,1241.0,1997.0,1161.0,245.0,343.0,2023.0
5,Almería,4343.0,2517.0,6437.0,4193.0,1106.0,1602.0,399.0,657.0,397.0,67.0,81.0,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,Zaragoza,5171.0,1988.0,14860.0,7904.0,1895.0,1756.0,1271.0,1531.0,1052.0,96.0,235.0,2010.0
780,Ceuta,833.0,393.0,983.0,723.0,497.0,93.0,49.0,189.0,141.0,8.0,13.0,2010.0
781,Melilla,1011.0,501.0,1043.0,774.0,427.0,156.0,79.0,194.0,171.0,1.0,8.0,2010.0
782,En el extranjero,158.0,17.0,2427.0,674.0,142.0,19.0,51.0,177.0,138.0,5.0,6.0,2010.0


In [9]:
data_coordinates_total = data_clean_years.copy()

data_coordinates_total = add_coordinates_from_dict(data_coordinates_total, provincias_coordenadas)
data_coordinates_total.head(10)

,Province,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos,Year,Latitude,Longitude
1,Total Nacional,226761.0,118125.0,665622.0,272638.0,104988.0,84721.0,35072.0,64711.0,44406.0,3844.0,8238.0,2023.0,NaN,NaN
2,Araba/Álava,1386.0,745.0,4184.0,1099.0,8.0,434.0,178.0,114.0,82.0,6.0,14.0,2023.0,42.84671,-2.67245
3,Albacete,1638.0,851.0,3771.0,1784.0,421.0,636.0,204.0,211.0,120.0,23.0,46.0,2023.0,38.99435,-1.85854
4,Alicante/Alacant,11416.0,5326.0,27014.0,12716.0,3723.0,6275.0,1241.0,1997.0,1161.0,245.0,343.0,2023.0,38.34517,-0.48149
5,Almería,4343.0,2517.0,6437.0,4193.0,1106.0,1602.0,399.0,657.0,397.0,67.0,81.0,2023.0,36.83405,-2.46371
6,Ávila,722.0,453.0,784.0,646.0,79.0,269.0,75.0,26.0,10.0,7.0,5.0,2023.0,40.65668,-4.68186
7,Badajoz,2516.0,1621.0,4844.0,2682.0,622.0,967.0,383.0,231.0,107.0,29.0,70.0,2023.0,38.87945,-6.97065
8,Balears (Illes),10159.0,4495.0,23009.0,6806.0,3195.0,1937.0,738.0,1519.0,994.0,82.0,228.0,2023.0,39.57119,2.64663
9,Barcelona,16470.0,10527.0,141789.0,44688.0,22066.0,12042.0,5442.0,22792.0,17358.0,808.0,1787.0,2023.0,41.38506,2.17340
10,Burgos,1393.0,755.0,2828.0,1532.0,319.0,588.0,271.0,183.0,113.0,10.0,36.0,2023.0,42.34399,-3.69691


In [10]:
data_coordinates_total.columns

Index(['Province', '1. CONTRA LAS PERSONAS', '1.2.-Lesiones', '5.1.-Hurtos',
       '5.2.-Robos con fuerza en las cosas',
       '5.2.1.-Robos con fuerza en las cosas en el interior de vehículos',
       '5.2.2.-Robos con fuerza en viviendas',
       '5.2.3.-Robos con fuerza en establecimientos',
       '5.3.-Robos con violencia o intimidación',
       '5.3.1.-Robos con violencia en vía pública',
       '5.3.2.-Robos con violencia en viviendas',
       '5.3.3.-Robos con violencia en establecimientos', 'Year', 'Latitude',
       'Longitude'],
      dtype='object')

In [11]:
# Crear un data_frame más ajustado para el mapa
# Primero, crear una columna de crímenes totales por provincia
data_total_crimes = data_coordinates_total.copy()
data_total_crimes['Total_crimes'] = data_coordinates_total[['1. CONTRA LAS PERSONAS', '1.2.-Lesiones', '5.1.-Hurtos',
       '5.2.-Robos con fuerza en las cosas',
       '5.2.1.-Robos con fuerza en las cosas en el interior de vehículos',
       '5.2.2.-Robos con fuerza en viviendas',
       '5.2.3.-Robos con fuerza en establecimientos',
       '5.3.-Robos con violencia o intimidación',
       '5.3.1.-Robos con violencia en vía pública',
       '5.3.2.-Robos con violencia en viviendas',
       '5.3.3.-Robos con violencia en establecimientos']].sum(axis = 1)

data_total_crimes

,Province,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos,Year,Latitude,Longitude,Total_crimes
1,Total Nacional,226761.0,118125.0,665622.0,272638.0,104988.0,84721.0,35072.0,64711.0,44406.0,3844.0,8238.0,2023.0,NaN,NaN,1629126.0
2,Araba/Álava,1386.0,745.0,4184.0,1099.0,8.0,434.0,178.0,114.0,82.0,6.0,14.0,2023.0,42.84671,-2.67245,8250.0
3,Albacete,1638.0,851.0,3771.0,1784.0,421.0,636.0,204.0,211.0,120.0,23.0,46.0,2023.0,38.99435,-1.85854,9705.0
4,Alicante/Alacant,11416.0,5326.0,27014.0,12716.0,3723.0,6275.0,1241.0,1997.0,1161.0,245.0,343.0,2023.0,38.34517,-0.48149,71457.0
5,Almería,4343.0,2517.0,6437.0,4193.0,1106.0,1602.0,399.0,657.0,397.0,67.0,81.0,2023.0,36.83405,-2.46371,21799.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,Zaragoza,5171.0,1988.0,14860.0,7904.0,1895.0,1756.0,1271.0,1531.0,1052.0,96.0,235.0,2010.0,41.64882,-0.88909,37759.0
780,Ceuta,833.0,393.0,983.0,723.0,497.0,93.0,49.0,189.0,141.0,8.0,13.0,2010.0,35.88939,-5.31979,3922.0
781,Melilla,1011.0,501.0,1043.0,774.0,427.0,156.0,79.0,194.0,171.0,1.0,8.0,2010.0,35.29298,-2.93871,4365.0
782,En el extranjero,158.0,17.0,2427.0,674.0,142.0,19.0,51.0,177.0,138.0,5.0,6.0,2010.0,NaN,NaN,3814.0


In [12]:
data_frame_total_map = data_total_crimes[['Province', 'Year', 'Latitude', 'Longitude', 'Total_crimes']]

data_frame_total_map

,Province,Year,Latitude,Longitude,Total_crimes
1,Total Nacional,2023.0,NaN,NaN,1629126.0
2,Araba/Álava,2023.0,42.84671,-2.67245,8250.0
3,Albacete,2023.0,38.99435,-1.85854,9705.0
4,Alicante/Alacant,2023.0,38.34517,-0.48149,71457.0
5,Almería,2023.0,36.83405,-2.46371,21799.0
...,...,...,...,...,...
779,Zaragoza,2010.0,41.64882,-0.88909,37759.0
780,Ceuta,2010.0,35.88939,-5.31979,3922.0
781,Melilla,2010.0,35.29298,-2.93871,4365.0
782,En el extranjero,2010.0,NaN,NaN,3814.0


In [18]:
values_to_remove = ['    Total Nacional', '    En el extranjero','    Desconocida']

data_map_filtered = drop_specific_rows(data_frame_total_map, 'Province', values_to_remove)


data_map_filtered['Year'] = data_map_filtered['Year'].fillna(0)

data_map_filtered['Year'] = data_map_filtered['Year'].astype(int)


data_map_filtered

C:\Users\VSPC\AppData\Local\Temp\ipykernel_27896\2342436721.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_map_filtered['Year'] = data_map_filtered['Year'].fillna(0)
C:\Users\VSPC\AppData\Local\Temp\ipykernel_27896\2342436721.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_map_filtered['Year'] = data_map_filtered['Year'].astype(int)


,Province,Year,Latitude,Longitude,Total_crimes
2,Araba/Álava,2023,42.84671,-2.67245,8250.0
3,Albacete,2023,38.99435,-1.85854,9705.0
4,Alicante/Alacant,2023,38.34517,-0.48149,71457.0
5,Almería,2023,36.83405,-2.46371,21799.0
6,Ávila,2023,40.65668,-4.68186,3076.0
...,...,...,...,...,...
777,Bizkaia,2010,43.26301,-2.93499,43587.0
778,Zamora,2010,41.50332,-5.74456,3575.0
779,Zaragoza,2010,41.64882,-0.88909,37759.0
780,Ceuta,2010,35.88939,-5.31979,3922.0


In [19]:
data_pivot_table = data_map_filtered.pivot_table(index=['Province', 'Latitude', 'Longitude'], columns='Year', values='Total_crimes', aggfunc='sum').reset_index()

data_pivot_table

Year,Province,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Albacete,38.99435,-1.85854,12849.0,12812.0,13103.0,12757.0,11743.0,10414.0,9375.0,9537.0,9467.0,9408.0,6407.0,6773.0,9307.0,9705.0
1,Alicante/Alacant,38.34517,-0.48149,108020.0,104511.0,101450.0,95120.0,88872.0,81636.0,79411.0,76329.0,71339.0,69361.0,49710.0,52437.0,63483.0,71457.0
2,Almería,36.83405,-2.46371,33407.0,34001.0,34064.0,31206.0,29297.0,27369.0,26329.0,24305.0,21349.0,20578.0,15370.0,17435.0,20129.0,21799.0
3,Araba/Álava,42.84671,-2.67245,13368.0,12647.0,11456.0,10880.0,10493.0,9376.0,8339.0,8204.0,8013.0,8027.0,5209.0,6202.0,8059.0,8250.0
4,Asturias,43.36191,-5.84939,25092.0,25153.0,26049.0,25055.0,23052.0,21103.0,19406.0,17874.0,17189.0,17178.0,12764.0,13765.0,17376.0,19612.0
5,Badajoz,38.87945,-6.97065,18685.0,17758.0,17563.0,18139.0,17472.0,15937.0,14637.0,14368.0,13126.0,12991.0,9937.0,10317.0,12508.0,14072.0
6,Balears (Illes),39.57119,2.64663,65555.0,69822.0,69785.0,67225.0,61365.0,56059.0,56131.0,58296.0,55746.0,55213.0,33635.0,39976.0,53417.0,53162.0
7,Barcelona,41.38506,2.17340,320008.0,309329.0,306689.0,286271.0,272121.0,265837.0,258590.0,269297.0,296763.0,310081.0,193738.0,232609.0,288419.0,295769.0
8,Bizkaia,43.26301,-2.93499,43587.0,43218.0,46440.0,46846.0,44193.0,39094.0,37792.0,37420.0,39616.0,40897.0,26097.0,31813.0,39873.0,37028.0
9,Burgos,42.34399,-3.69691,12225.0,11971.0,11291.0,10338.0,10251.0,8920.0,7996.0,7941.0,7711.0,7768.0,5518.0,6847.0,7632.0,8028.0


In [15]:
data_pivot_table.to_csv('data_pivot_table.csv', index = False)